In [17]:
from config import BaselineCNNconfig, SegmentedCNNconfig, TpCNNconfig, AugerinoCNNconfig
from tpreporter import Reporter
from model_fitter import ModelFitter
import argparse
import torch
from datetime import date

In [2]:
model_configs = {
    'baseline': BaselineCNNconfig,
    'segmented': SegmentedCNNconfig,
    'tp-cnn': TpCNNconfig,
    'augerino': AugerinoCNNconfig,
}

In [6]:
class Args:
    def __init__(self):
        self.config = 'baseline'
        self.data_path = 'data/GTZAN'
        self.local = False
        self.checkpoint = 'checkpoints'
        self.test_only = False
args = Args()

In [9]:
config_name = args.config
if config_name is None:
    raise ValueError('No config name passed to script!')
print('Grabbing model_config')
config = model_configs[config_name]
print(f'Using config {config_name}, local: ${args.local}')

Grabbing model_config
Using config baseline, local: $False


In [10]:
use_cuda = torch.cuda.is_available() and not args.local
torch.manual_seed(config.seed)
device = torch.device("cuda" if use_cuda else "cpu")
print(f'Using device {device}')

Using device cpu


In [11]:
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}

save_directory = f'{args.checkpoint}/{date.today().strftime("%d-%m-%Y")}'

print(f'Save directory: {save_directory}')

Save directory: checkpoints/10-02-2021


In [12]:
reporter = Reporter

model_fitter = ModelFitter(args, config, device, kwargs, reporter)

In [21]:
import pandas as pd
import copy

def get_data_frame(data_path, is_local):
    temp_df = None
    if is_local:
        temp_df = pd.read_csv(f"{data_path}/test.csv")
    else:
        temp_df = pd.read_csv(f"{data_path}/features_30_sec.csv")
    temp_df['filePath'] = f"{data_path}/genres_original/{temp_df['label']}/{temp_df['filename']}"
    print(temp_df['label'])
    ids = copy.deepcopy(temp_df['filename'])

    for index, id in enumerate(ids):
        bits = id.split('.')
        ids[index] = f"id-{bits[0][0:2]}{bits[1]}-original"
    temp_df['ID'] = ids

    return temp_df.loc[:, ['ID','filePath', 'label']]
df = get_data_frame('data/GTZAN', False)
df.head()

,ID,filePath,label
0,id-bl00000-original,data/GTZAN/genres_original/0 blues\n1 ...,blues
1,id-bl00001-original,data/GTZAN/genres_original/0 blues\n1 ...,blues
2,id-bl00002-original,data/GTZAN/genres_original/0 blues\n1 ...,blues
3,id-bl00003-original,data/GTZAN/genres_original/0 blues\n1 ...,blues
4,id-bl00004-original,data/GTZAN/genres_original/0 blues\n1 ...,blues


In [18]:
if not args.test_only:
    model_fitter.fit()

Loading in data...
Preparing test values...


FileNotFoundError: [Errno 2] No such file or directory: 'data/GTZAN/genres_original/0      blues\n1      blues\n2      blues\n3      blues\n4      blues\n       ...  \n995     rock\n996     rock\n997     rock\n998     rock\n999     rock\nName: label, Length: 1000, dtype: object/0      blues.00000.wav\n1      blues.00001.wav\n2      blues.00002.wav\n3      blues.00003.wav\n4      blues.00004.wav\n            ...       \n995     rock.00095.wav\n996     rock.00096.wav\n997     rock.00097.wav\n998     rock.00098.wav\n999     rock.00099.wav\nName: filename, Length: 1000, dtype: object'